In [1]:
!pip install -q langchain langgraph openai


[notice] A new release of pip is available: 23.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
!pip install -q langchain_community


[notice] A new release of pip is available: 23.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
!pip install -q langchain_openai


[notice] A new release of pip is available: 23.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:

from langgraph.graph import StateGraph,START,END
from typing_extensions import TypedDict
from typing import Annotated
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph.message import add_messages
from langchain_core.messages import HumanMessage, AIMessage
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableLambda #to wrap python funcs into a runnable chain for langraph nodes


In [5]:

from langchain_core.messages import BaseMessage

class Study(TypedDict):
    messages: Annotated[list[BaseMessage], add_messages]
    topic: str
    age: int
    style: str


In [6]:

import os
os.environ["OPENAI_API_KEY"] = "YOUR_API_KEY_HERE"


In [7]:
llm=ChatOpenAI(model="gpt-4",temperature=0.7)

In [8]:
summary_prompt=ChatPromptTemplate.from_messages([("system","Summarise this textbook content in simple words for person aged {age}"),
("human","{topic}")])

In [9]:
async def summarizer_node(state: Study) -> Study:
    print("\n Summarizer Node Triggered!")
    print(" Topic:", state["topic"])
    print(" Age:", state["age"])
    
    runnable_chain = summary_prompt | llm
    try:
        response = await runnable_chain.ainvoke({
            "topic": state["topic"],
            "age": state["age"]
        })
        print("Got summary response:", response)
        return {
            "messages": state["messages"] + [AIMessage(content=response.content)],
            "topic": state["topic"],
            "age": state["age"],
            "style": state["style"]
        }
    except Exception as e:
        print(" Error in summarizer_node:", e)
        return state  # fallback


In [40]:
quiz_prompt=ChatPromptTemplate.from_messages([("system","Make a fun 5 quiz with questions and answers from this summary for person aged {age}. Format each as:\nQ1 Q2 ...\nA1 A2 ..."),
                                               ("human","{summary}")
                                                ])


In [14]:
async def quiz_node(state: Study) -> Study:
    print("\n Quiz Node Triggered!")
    
    summary = state["messages"][-1].content if state["messages"] else "MISSING"
    print(" Summary being passed:", summary)

    runnable_chain = quiz_prompt | llm
    try:
        quiz = await runnable_chain.ainvoke({
            "summary": summary,
            "age": state["age"]
        })
        print(" Got quiz response:", quiz)

        new_state = {
            "messages": state["messages"] + [AIMessage(content=quiz.content)],
            "topic": state["topic"],
            "age": state["age"],
            "style": state["style"]
        }

        print(" Returning from quiz_node with messages:")
        for msg in new_state["messages"]:
            print(f" - Type: {type(msg)}, Content: {msg.content}")

        return new_state

    except Exception as e:
        print(" Error in quiz_node:", e)
        return state


In [15]:
print("Summarizer Node:", summarizer_node)
print("Quiz Node:", quiz_node)


Summarizer Node: <function summarizer_node at 0x0000022E9E5E96C0>
Quiz Node: <function quiz_node at 0x0000022E9D114E50>


In [16]:
puzzle_prompt = ChatPromptTemplate.from_messages([
    ("system", "You're a puzzle maker for people aged {age}. Turn the summary into a mystery or logic puzzle with answers.Format as:\nP: ...\nA"),
    ("human", "{summary}")
])


In [17]:
async def puzzle_node(state: Study) -> Study:
    print("\n Puzzle Node Triggered!")

    summary = state["messages"][-1].content if state["messages"] else "MISSING"
    print(" Summary being passed:", summary)

    runnable_chain = puzzle_prompt | llm
    try:
        puzzle = await runnable_chain.ainvoke({
            "summary": summary,
            "age": state["age"]
        })
        print(" Got puzzle response:", puzzle)

        new_state = {
            "messages": state["messages"] + [AIMessage(content=puzzle.content)],
            "topic": state["topic"],
            "age": state["age"],
            "style": state["style"]
        }

        print(" Returning from puzzle_node with messages:")
        for msg in new_state["messages"]:
            print(f" - Type: {type(msg)}, Content: {msg.content}")

        return new_state

    except Exception as e:
        print("Error in puzzle_node:", e)
        return state


In [18]:
def choose_game(state: Study) -> str:
    if state.get("style") == "quiz":
        return "quiz"
    else:
        return "puzzle"

In [19]:
graph = StateGraph(Study)
graph.add_node("summarize", summarizer_node)
graph.add_node("quiz", quiz_node)
graph.add_node("puzzle", puzzle_node)  

graph.set_entry_point("summarize")
graph.add_conditional_edges("summarize", choose_game, {
    "quiz": "quiz",
    "puzzle": "puzzle"
})
graph.add_edge("quiz", END)
graph.add_edge("puzzle", END)

compiled_graph = graph.compile(checkpointer=MemorySaver())


In [54]:
import asyncio
import nest_asyncio
nest_asyncio.apply()

"""
async def run_study(topic: str, age: int = 12, style: str = "quiz"):
    initial_state = {"topic": topic, "age": age, "style": style, "messages": []}
    config = {"configurable": {"thread_id": "study-thread-" + str(hash(topic + str(age) + style))}}

    print(f"\nStudy Game: {topic} (Age {age}, Style: {style})")
    print("─" * 55)

    try:
        async for step in compiled_graph.astream(initial_state, config):
            if "messages" in step:
                for msg in step["messages"]:
                    if isinstance(msg, AIMessage):
                        print(f"\n {msg.content.strip()}")
            if "__end__" in step:
                print("\nThat’s the end of the game ")
    except Exception as e:
        print("Something went wrong:", e)


import asyncio
import nest_asyncio
import re
nest_asyncio.apply()
"""
import asyncio
import nest_asyncio
import re
from langchain_core.messages import AIMessage
nest_asyncio.apply()
import asyncio
import nest_asyncio
import re
from langchain_core.messages import AIMessage

nest_asyncio.apply()

async def run_study(topic: str, age: int = 12, style: str = "quiz"):
    initial_state = {"topic": topic, "age": age, "style": style, "messages": []}
    config = {"configurable": {"thread_id": f"study-thread-{hash(topic + str(age) + style)}"}}

    print(f"\nStudy Game: {topic} (Age {age}, Style: {style})")
    print("─" * 60)

    all_questions = []
    all_answers = []

    try:
        async for step in compiled_graph.astream(initial_state, config):
            if "messages" in step:
                for msg in step["messages"]:
                    if not isinstance(msg, AIMessage):
                        continue

                    content = msg.content.strip()

                    # Extract Q&A pairs
                    if style == "quiz" and "Q:" in content and "A:" in content:
                        qa_pairs = re.findall(r"Q:\s*(.*?)\nA:\s*(.*?)(?=\nQ:|\Z)", content, re.DOTALL)
                        for q, a in qa_pairs:
                            all_questions.append(q.strip())
                            all_answers.append(a.strip())

                    elif style == "puzzle" and "P:" in content and "A:" in content:
                        puzzle_pairs = re.findall(r"P:\s*(.*?)\nA:\s*(.*?)(?=\nP:|\Z)", content, re.DOTALL)
                        for p, a in puzzle_pairs:
                            all_questions.append(p.strip())
                            all_answers.append(a.strip())

                    else:
                        print(f"\n {content}\n---")

            elif "__end__" in step:
                print("\nStudy session complete.")

    except Exception as e:
        print("Something went wrong:", e)


In [55]:


await run_study("What is process of madhubani painting", age=7, style="puzzle")



Study Game: What is process of madhubani painting (Age 7, Style: puzzle)
────────────────────────────────────────────────────────────

 Summarizer Node Triggered!
 Topic: What is process of madhubani painting
 Age: 7
Got summary response: content="Madhubani painting is a cool art form from India. It is a way of drawing and coloring that starts with making a rough sketch using a pencil. Then, the artist goes over that with a pen, using special black ink. The picture is then filled in with bright colors like red, blue, green, and yellow. The pictures often show things like sun, moon, trees, animals, and people. This kind of art is really special because it tells stories and traditions from the artist's culture." additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 102, 'prompt_tokens': 33, 'total_tokens': 135, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, '

In [56]:


await run_study("Explain the tea leaf paradox", age=16, style="quiz")



Study Game: Explain the tea leaf paradox (Age 16, Style: quiz)
────────────────────────────────────────────────────────────

 Summarizer Node Triggered!
 Topic: Explain the tea leaf paradox
 Age: 16
Got summary response: content="The tea leaf paradox is a phenomenon that you might have noticed when you stir a cup of tea with leaves at the bottom. When you stir the tea, instead of being pushed to the sides of the cup due to centrifugal force, the leaves gather in the center. This happens because stirring creates a whirlpool effect where the surface of the liquid is higher at the edges and lower in the middle. The higher pressure at the edges pushes the leaves towards the center. It's not really a paradox, but it’s called so because it seems to go against what we'd initially expect." additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 117, 'prompt_tokens': 31, 'total_tokens': 148, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'aud